# Crawl CNA news from several categories

Only 5 pieces of news on the first page from each category

中央社新聞各類新聞各5篇


In [16]:
import requests
from bs4 import BeautifulSoup
import re

# There are 11 categories in CNA news

Crawl links on a category's first page.

中央社新聞爬蟲-11類別的新聞(只能爬第一頁的內容，沒有翻頁的功能)

In [17]:
!pip install fake_useragent


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Crawl news for 11 categories

In [18]:
# 11 news categories
#news_links =['aipl', 'ait', 'aspt', 'asc', 'aie', 'amov','ahel','aopl','asoc','acul','acn']
#news_categories=['政治','科技','運動','證卷','產經','娛樂','生活','國際','社會','文化','兩岸']
news_links =['aipl', 'ait', 'aspt']
news_categories=['政治','科技','運動']
base_url = 'https://www.cna.com.tw/list/'

In [19]:
import requests
import re
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime, timedelta
from fake_useragent import UserAgent

In [20]:
user_agent = UserAgent()

In [21]:
%%time
# 存放資料之變數
links = []
titles = []
dates = []
contents = []
categories = []
item_id = []
photo_links = []


# 這裡我們需要用for迴圈爬取11個類別
# Here we need to crawl 11 categories using for loop
for i, url_short_name in enumerate(news_links):  #針對每一類 共有11類

    category = news_categories[i] #'政治' #類別名稱紀錄起來 

    # Categorical url link
    category_url = base_url + url_short_name + '.aspx'
    print("Getting categorical news:", category)

    # Request the categorical news page
    # req = requests.get(category_url)
    req = requests.get(category_url, headers={ 'user-agent': user_agent.random }, timeout=5)
    page = BeautifulSoup(req.text, 'lxml')

    # 抓新聞列表
    items = page.find('ul', {'id': re.compile("jsMainList\w*")}).findAll('li')
    #print(items)


    # Let's start to crawl the news in the first page for that category
    serial_no = 1
    for item_j, item in enumerate(items,start=1): #針對每一篇項目 抓其細節
        title = item.find('h2').text
        print(serial_no,'--', title )
        
        link = item.find('a').get('href')
        print(link)

        try:
            if item.find('img').has_attr('data-src'):
                photo_link = item.find('img').get('data-src')
            else:
                photo_link = item.find('img').get('src')
        except:
            photo_link = ''
        
        # get date and convert it with the format of  "2022-04-20"
        news_date = item.find('div', {'class':"date"}).text
        print(news_date)
        dtime = datetime.strptime(news_date, '%Y/%m/%d %H:%M')
        # conver to the format of  "2022-04-20"
        news_date_str = dtime.strftime("%Y-%m-%d") # keep only date, remove the time.

        categories.append(category)
        titles.append(title)
        links.append(link)
        photo_links.append(photo_link)
        dates.append(news_date_str)
 
        # item_id.append("cna_" + tstr + "_" + str(serial_no))
        # conver to the format of  "20220420"
        dtstr = dtime.strftime("%Y%m%d") # keep only date, remove the time.
        # format of serial number: aipl_20220314_1
        
        item_id.append(url_short_name + "_" + dtstr + "_" + str(serial_no))
        serial_no += 1

        #抓新聞內容
        # 最新的新聞列表之新聞連結:開頭缺少"https://www.cna.com.tw"
        if link.startswith("/news"):
            link = "https://www.cna.com.tw"+link
        
        print(link)
        req = requests.get(link, headers={ 'user-agent': user_agent.random }, timeout=5)
        
        page = BeautifulSoup(req.text,'lxml')

        # Find content
        content = page.find('div',{'class':"paragraph"})

        # Remove unnecessary content
        # There are some words we don't like to analyze, They should be removed.
        # Remove "透過 Google News追蹤中央社"
        try:
            content.find('div', {'class':"gmailNews"}).decompose()
        except:
            pass
        # 額外插入的延伸閱讀區塊-->移除之
        try:
            content.find('div', {'class':"paragraph moreArticle"}).decompose()
        except:
            pass
        # 額外插入的訂閱連結-->移除之
        try:
            content.find('div', class_="SubscriptionInner mySubscriptionInner").decompose()
        except:
            pass

        # 額外插入的圖片與文字說明區塊-->移除之
        try:
            for media in content.findAll('div', {'class':"media"}):
                media.decompose()
        except:
            pass

        # Remove unnecessary content
        cont = content.text
        cont = re.sub('（中央社.+?）','', cont)
        cont = re.sub('（編輯：.*','', cont) 
        cont = re.sub('（譯者：.*','', cont) 
        
        contents.append(cont)

        if item_j >= 4: # Here we crawl only 4 pieces of news for each category, in order to save time.
            break 
        
        time.sleep(10)  # 遵守爬蟲禮節，請小睡一下


Getting categorical news: 政治
1 -- 美國務院指中國曲解聯大2758號決議 外交部誠摯感謝
/news/aipl/202503080218.aspx
2025/03/08 20:50
https://www.cna.com.tw/news/aipl/202503080218.aspx
2 -- 解析川普「4R」 林佳龍：台灣是非紅供應鏈關鍵角色
/news/aipl/202503080214.aspx
2025/03/08 20:37
https://www.cna.com.tw/news/aipl/202503080214.aspx
3 -- 民進黨政策宣講15日台中首發場 定調對抗混亂在野
/news/aipl/202503083001.aspx
2025/03/08 20:22
https://www.cna.com.tw/news/aipl/202503083001.aspx
4 -- 藍推反廢死公投 黃國昌：立院合作開放找時間詳談
/news/aipl/202503080200.aspx
2025/03/08 19:32
https://www.cna.com.tw/news/aipl/202503080200.aspx
Getting categorical news: 科技
1 -- SpaceX星艦第8次試飛又爆炸 碎片影響佛州飛航[影]
/news/ait/202503070073.aspx
2025/03/07 11:02
https://www.cna.com.tw/news/ait/202503070073.aspx
2 -- 培養科技女力 國科會啟動2025 GiCS尋找資安女婕思
/news/ait/202503080155.aspx
2025/03/08 16:54
https://www.cna.com.tw/news/ait/202503080155.aspx
3 -- 美民間企業二度嘗試登月 登陸器再次傾倒以側面著陸
/news/ait/202503070201.aspx
2025/03/07 16:20
https://www.cna.com.tw/news/ait/202503070201.aspx
4 -- 中國加強碳排控管 鋼鋁水泥業年底納入碳市場
https://netzero.cna.co

In [22]:
titles

['美國務院指中國曲解聯大2758號決議 外交部誠摯感謝',
 '解析川普「4R」 林佳龍：台灣是非紅供應鏈關鍵角色',
 '民進黨政策宣講15日台中首發場 定調對抗混亂在野',
 '藍推反廢死公投 黃國昌：立院合作開放找時間詳談',
 'SpaceX星艦第8次試飛又爆炸 碎片影響佛州飛航[影]',
 '培養科技女力 國科會啟動2025 GiCS尋找資安女婕思',
 '美民間企業二度嘗試登月 登陸器再次傾倒以側面著陸',
 '中國加強碳排控管 鋼鋁水泥業年底納入碳市場',
 '響應FIFA培育人才計畫 高雄先鋒女足將成立預備軍',
 'PLG林志傑復出得15分  勇士擊退獵鷹',
 'HBL北一女斷淡商小巨蛋12連勝 晉冠軍戰拚衛冕',
 'HBL松山劉程恩罰球大心臟 感謝隊友一起逆轉勝']

In [23]:
contents

['美國國務院昨天指出，中國刻意濫用和扭曲聯大2758決議，是中國施壓行動的一環，意在將台灣孤立於國際社會。外交部今天表示，美國充分彰顯歷來對此議題所採取的一貫堅定立場，對此表達誠摯感謝。中國外交部長王毅7日稱，聯合國大會第2758號決議已徹底解決包括台灣在內全中國在聯合國的代表權問題，台灣在聯合國的唯一稱謂就是「中國台灣省」，台灣從來不是一個國家，過去不是，今後更絕無可能。美國國務院一名發言人美東時間7日對中央社指出，刻意誤用和曲解聯合國大會第2758號決議，是中國施壓行動一環，目的是使台灣孤立於國際社會。外交部表示，聯大第2758號決議僅決定中國在聯合國代表權的歸屬，全文未曾提及台灣，也未認定台灣是中華人民共和國的一部分，更未授權中華人民共和國在聯合國代表台灣。外交部指出，部長林佳龍已多次在各種不同場合強調前述客觀事實，尤其指出聯大第2758號決議並未涉及或排除台灣參加國際組織的權利，這也是許多民主國家的共識；除美國外，英國、瑞典、荷蘭、澳大利亞、加拿大、匈牙利、比利時、瓜地馬拉等國的行政或立法部門，以及歐盟執委會與歐洲議會等，都已公開表示相同的立場，並力挺台灣。外交部呼籲中方認清現實，台灣是主權獨立國家，與中華人民共和國互不隸屬，有權與各國交往，更何況參與國際組織是全體台灣人民的正當權利，中國無權置喙。外交部指出，對於中國企圖將台灣問題「內政化」，片面改變台海現狀，為其未來武力犯台建構法理基礎的野心，予以嚴正譴責，並呼籲國際社會持續以具體行動加以反制，駁斥中國所謂「一中原則」已是國際共識的謬論，台灣將繼續與理念相近國家共同合作，維護台海及印太地區的和平、穩定及繁榮。',
 '外交部長林佳龍指出，美國總統川普的性格特質可透過「4R」來理解，包括Reciprocity（互惠）、Reindustrialization（再工業化）、Respect（尊重）及Real Estate（不動產），而台灣掌握民主國家的信賴與技術，在非紅供應鏈中扮演關鍵角色。林佳龍今天在臉書發文指出，昨天應邀赴風傳媒「大浪當前－2025高峰論壇」，致詞分析川普（Donald Trump）的施政理念與台灣的因應之道。林佳龍表示，川普在安全領域將延續印太戰略，並透過美日峰會、Quad外長會議及美日韓外長會議進行再確認，但與第一任期相比，川普更強調「盟友責任分擔」與「安全靠實力」，包括台灣、日本、菲律

In [24]:
item_id

['aipl_20250308_1',
 'aipl_20250308_2',
 'aipl_20250308_3',
 'aipl_20250308_4',
 'ait_20250307_1',
 'ait_20250308_2',
 'ait_20250307_3',
 'ait_20250307_4',
 'aspt_20250308_1',
 'aspt_20250308_2',
 'aspt_20250308_3',
 'aspt_20250308_4']

# Save data

In [25]:
data = zip(item_id, dates, categories, titles, contents, links, photo_links)
df = pd.DataFrame(list(data), columns=['item_id','date','category','title','content','link','photo_link'])

In [26]:
df.head(2)

,item_id,date,category,title,content,link,photo_link
0,aipl_20250308_1,2025-03-08,政治,美國務院指中國曲解聯大2758號決議 外交部誠摯感謝,美國國務院昨天指出，中國刻意濫用和扭曲聯大2758決議，是中國施壓行動的一環，意在將台灣孤立...,/news/aipl/202503080218.aspx,https://imgcdn.cna.com.tw/www/WebPhotos/200/20...
1,aipl_20250308_2,2025-03-08,政治,解析川普「4R」 林佳龍：台灣是非紅供應鏈關鍵角色,外交部長林佳龍指出，美國總統川普的性格特質可透過「4R」來理解，包括Reciprocity（...,/news/aipl/202503080214.aspx,https://imgcdn.cna.com.tw/www/WebPhotos/200/20...


In [27]:
df.shape

(12, 7)

In [28]:
df.content[0]

'美國國務院昨天指出，中國刻意濫用和扭曲聯大2758決議，是中國施壓行動的一環，意在將台灣孤立於國際社會。外交部今天表示，美國充分彰顯歷來對此議題所採取的一貫堅定立場，對此表達誠摯感謝。中國外交部長王毅7日稱，聯合國大會第2758號決議已徹底解決包括台灣在內全中國在聯合國的代表權問題，台灣在聯合國的唯一稱謂就是「中國台灣省」，台灣從來不是一個國家，過去不是，今後更絕無可能。美國國務院一名發言人美東時間7日對中央社指出，刻意誤用和曲解聯合國大會第2758號決議，是中國施壓行動一環，目的是使台灣孤立於國際社會。外交部表示，聯大第2758號決議僅決定中國在聯合國代表權的歸屬，全文未曾提及台灣，也未認定台灣是中華人民共和國的一部分，更未授權中華人民共和國在聯合國代表台灣。外交部指出，部長林佳龍已多次在各種不同場合強調前述客觀事實，尤其指出聯大第2758號決議並未涉及或排除台灣參加國際組織的權利，這也是許多民主國家的共識；除美國外，英國、瑞典、荷蘭、澳大利亞、加拿大、匈牙利、比利時、瓜地馬拉等國的行政或立法部門，以及歐盟執委會與歐洲議會等，都已公開表示相同的立場，並力挺台灣。外交部呼籲中方認清現實，台灣是主權獨立國家，與中華人民共和國互不隸屬，有權與各國交往，更何況參與國際組織是全體台灣人民的正當權利，中國無權置喙。外交部指出，對於中國企圖將台灣問題「內政化」，片面改變台海現狀，為其未來武力犯台建構法理基礎的野心，予以嚴正譴責，並呼籲國際社會持續以具體行動加以反制，駁斥中國所謂「一中原則」已是國際共識的謬論，台灣將繼續與理念相近國家共同合作，維護台海及印太地區的和平、穩定及繁榮。'

In [29]:
df.to_csv("cna_category_news.csv", sep="|", index=False)

In [30]:
photo_links

['https://imgcdn.cna.com.tw/www/WebPhotos/200/20250308/1200x744_236214943764.jpg',
 'https://imgcdn.cna.com.tw/www/WebPhotos/200/20250308/1200x775_265738111916.jpg',
 'https://imgcdn.cna.com.tw/www/WebPhotos/200/20240419/1984x1488_wmky_27799879179_0.jpg',
 'https://imgcdn.cna.com.tw/www/WebPhotos/200/20250308/1152x768_wmkn_0_C20250308000154.jpg',
 'https://imgcdn.cna.com.tw/www/webphotos/WebCover/420/20250307/800x600_689221744159.jpg',
 '',
 '',
 '',
 'https://imgcdn.cna.com.tw/www/WebPhotos/200/20250308/1426x768_wmkn_0_C20250308000173.jpg',
 'https://imgcdn.cna.com.tw/www/WebPhotos/200/20250308/1024x768_wmkn_0_C20250308000157.jpg',
 'https://imgcdn.cna.com.tw/www/WebPhotos/200/20250308/1200x837_wmky_936923128254_202503080150000000.jpg',
 'https://imgcdn.cna.com.tw/www/WebPhotos/200/20250308/1200x981_wmky_959023812487_202503080120000000.jpg']